Preparing Data

<p> We will be combining data from three files </p>
<ol>
<li> We will select name, artist, genre, and number of grammy awards from grammySongs_1999-2019.csv. This information is not immediately available from the sheet, so we will group by artist and name and count the number of grammys each song has won. </li>
<li> We will select artist, name, peak position, average position, and weeks on chart from billboardHot100_1999-2019. </li>
<li> We will select artist, name, energy, danceability, tempo, and valence from data/songAtrributes_1999-2019.csv. </li>
</ol>

In [1]:
import pandas as pd

In [2]:
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
# grammyAwards = grammyAwards[['Name', 'Artist', 'Genre', 'GrammyAward']]
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist']).count().reset_index()
print(grammyAwards.size)
grammyAwards.head()

1098


,Name,Artist,GrammyAward
0,'Til Summer Comes Around,Keith Urban,1
1,24 Frames,"Ason Isbell, Songwriter",1
2,24k Magic,Bruno Mars,1
3,500 Miles High,"Chick Corea, Soloist",1
4,99 Problems,Jay-z,1


In [3]:
billboardHot100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
billboardHot100 = billboardHot100[['Artists', 'Name', 'Peak.position', 'Weeks.on.chart']]
billboardHot100['Weeks.on.chart'] = billboardHot100['Weeks.on.chart'].fillna(1)
billboardHot100['Peak.position'] = billboardHot100['Peak.position'].fillna(100)
# billboardHot100.groupby(by=['Artist', 'Name']).min('Peak.position').max('Weeks.on.chart')
billboardHot100 = billboardHot100.groupby(['Artists', 'Name']).agg({'Peak.position':'min', 'Weeks.on.chart':'max'})[['Peak.position','Weeks.on.chart']].reset_index()

In [5]:
songAtrributes = pd.read_csv('data/songAttributes_1999-2019.csv')
songAtrributes = songAtrributes[['Name', 'Energy', 'Danceability', 'Tempo', 'Valence']]

,Name,Energy,Danceability,Tempo,Valence
0,Welcome All Again,0.904,0.520,106.022,0.365
1,Fuzzy,0.709,0.581,120.027,0.408
2,Dig,0.918,0.572,144.061,0.370
3,You,0.661,0.596,111.975,0.183
4,My Days,0.808,0.520,92.721,0.666
...,...,...,...,...,...
154926,Nothing On You (feat. Paulo Londra & Dave),0.649,0.747,88.011,0.840
154927,I Don't Want Your Money (feat. H.E.R.),0.631,0.469,91.418,0.763
154928,1000 Nights (feat. Meek Mill & A Boogie Wit da...,0.590,0.778,97.995,0.515
154929,Way To Break My Heart (feat. Skrillex),0.731,0.726,170.125,0.430
